## How to run the script

Don't need to do anything, just press run.

## Purpose of the script

In this script we're looking at the trainning delivered from 1st April 2022 for the work group sets we've made recommendations for 

## 1. Get all file paths and read files

#### 1.A. Import relavent libraries

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x) # Supress scientific notation
import numpy as np
from azure.storage.blob import BlobServiceClient
import os
import pyodbc
from io import StringIO, BytesIO
import io


#### 1.B. Define connections to SQL tables and Azure Storage blobs

In [18]:
# Define the connection to SQL database
#conn = pyodbc.connect( # SQL server
                       # name of SQL data base
                        # UID =
                       # Password
#                      ) # databasename

# To connect to Azure devp envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

# To connect to Azure test envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

# To connect to Azure staging envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

# To connect to Azure prod envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)


#### 1.C. Define all the Functions used in the script

In [31]:
# Define a funtion to read csv files directly from azure data storage
def download_csv(BLOBNAME):
    blob_client_instance = blob_service_client_instance_devp.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)
    blob_data = blob_client_instance.download_blob()
    with BytesIO() as f:
        blob_data.readinto(f)
        f.seek(0)
        data = pd.read_csv(f)   
    return data

# Define a funtion to read excel files directly from azure data storage
def download_excel(BLOBNAME):
    blob_client_instance = blob_service_client_instance_devp.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)
    blob_data = blob_client_instance.download_blob()
    with BytesIO() as f:
        blob_data.readinto(f)
        f.seek(0)
        data = pd.read_excel(f)   
    return data

# Rename columns
def rename_column(table_name):
    table_name.columns = table_name.columns.str.replace(' ', '')
    table_name.columns = table_name.columns.str.lower()

# Save csv files and upload to azure blob storage
def save_outputs_azure_blobs(output_name1,output_name2,output_name3,azure_environment):
    output_name1.to_csv(output_name2, index = False)
    # Create a blob client using the local file name as the name for the blob
    blob_client = azure_environment.get_blob_client(container='input', blob='static/benefitsBaseline/{tmp2}/{tmp3}'.format(tmp2=output_name3,tmp3=output_name2))
    # Upload the created file
    with open(output_name2, "rb") as data:
        blob_client.upload_blob(data,overwrite=True)

In [4]:
# Recommendations LOSC (AZURE BLOB)
losc_recom = download_csv('static/benefitsBaseline/recommendations_sep2021_fix_June2022/losc_benefits_and_recommendations_2021_dcs_MAX.csv')
rename_column(losc_recom)

# Recommendations FMS
fms_recom = download_csv('static/benefitsBaseline/recommendations_sep2021_fix_June2022/fms_benefits_and_recommendations_2021_dcs.csv')
rename_column(fms_recom)

# Recommendations Workbank
workbank_recom = download_csv('static/benefitsBaseline/recommendations_sep2021_fix_June2022/workbank_benefits_and_recommendations_2021_dcs.csv')
rename_column(workbank_recom)

#### 1.D. Read the paths to all files/download blobs/read SQL tables

In [7]:
# TRAINNING DEMAND (SQL)
trainning_demand  = pd.read_sql_query('''select * from [dbo].[trainingDemand]''', conn)
rename_column(trainning_demand)

# ORGPLUS PRE INTERVENTION (SQL)
ORG_DATA_pre= pd.read_sql_query('''select * from [dbo].[orgPlusBaseline]''', conn)
rename_column(ORG_DATA_pre)
# ORGPLUS POST INTERVENTION (SQL)
ORG_DATA_post= pd.read_sql_query('''select * from [dbo].[orgPlus]''', conn)
rename_column(ORG_DATA_post)

# WGS PRE INTERVENTION (SQL)
workgroup_data_pre= pd.read_sql_query('''select * from [dbo].[workGroupSetBaseline]''', conn)
rename_column(workgroup_data_pre)
# WGS POST INTERVENTION (SQL)
workgroup_data_post= pd.read_sql_query('''select * from [dbo].[workGroupSet]''', conn)
rename_column(workgroup_data_post)

# COURSE LIST (SQL)
course_list= pd.read_sql_query('''select * from [dbo].[courseList]''', conn)
rename_column(course_list)

# Recommendations LOSC (AZURE BLOB)
losc_recom = download_csv('static/benefitsBaseline/recommendations_sep2021_fix_June2022/losc_benefits_and_recommendations_2021_dcs_MAX.csv')
rename_column(losc_recom)

# Recommendations FMS
fms_recom = download_csv('static/benefitsBaseline/recommendations_sep2021_fix_June2022/fms_benefits_and_recommendations_2021_dcs.csv')
rename_column(fms_recom)

# Recommendations Workbank
workbank_recom = download_csv('static/benefitsBaseline/recommendations_sep2021_fix_June2022/workbank_benefits_and_recommendations_2021_dcs.csv')
rename_column(workbank_recom)


## 2. Pre Process data

#### 2.A. Trainning demand

In [8]:
#Trainning demand drop empty columns
trainning_demand.dropna(how='all', axis=1, inplace=True)

## Filter out for trainning delievered
print(trainning_demand['enrollstatusbreakdown'].value_counts())
print(len(trainning_demand['enrollstatusbreakdown']))

trainning_demand = trainning_demand[['coursecategory','coursetitle','employeenumber','deliveryunit',
                                     'enrollstatusbreakdown','ced','csd','orgcostcentre','periodstart','startyear','periodend',
                                     'endyear','period']]
len(trainning_demand)

# Only keep trainnig where they've passed
trainning_demand = trainning_demand.loc[trainning_demand['enrollstatusbreakdown'].isin(['Completed', 'Passed', 'Attended'])]
print(trainning_demand['enrollstatusbreakdown'].value_counts())
len(trainning_demand)

# Only keep trainning delivered after 1st April 2022
trainning_demand['csd'] = pd.to_datetime(trainning_demand['csd'], format='%Y-%m-%d')
trainning_demand_filter = trainning_demand.loc[(trainning_demand['csd'] >= '2022-04-01')]


Place Booked                                 24958
Completed                                    16151
Passed                                       12123
Cancelled - replaced                          3421
Attended                                      3321
Cancelled - not required                      2603
Cancelled - date or location not suitable     2498
Booked - No Show                              2488
Cancelled - by training provider              1212
Cancelled - annual leave                      1046
Cancelled - illness or family issue            673
Cancelled - required at work                   603
Cancelled - no charge                          598
Cancelled - not eligible                       595
Failed                                         480
Cancelled - By Training Admin                  300
Attended - Appreciation                        239
Cancelled - other training commitments         239
Attended - Did Not Complete                    197
Partial Pass                   

#### 2.B. Orgplus

In [9]:
# OrgPlus
ORG_DATA_pre = ORG_DATA_pre.loc[~ORG_DATA_pre['employeenumber'].isna()]
ORG_DATA_pre['employeenumber'] = ORG_DATA_pre['employeenumber'].astype(int)
# Drop all columns which are blanks
ORG_DATA_pre.dropna(how='all', axis=1, inplace=True)
print(len(ORG_DATA_pre))

# OrgPlus
ORG_DATA_post = ORG_DATA_post.loc[~ORG_DATA_post['employeenumber'].isna()]
ORG_DATA_post['employeenumber'] = ORG_DATA_post['employeenumber'].astype(int)
# Drop all columns which are blanks
ORG_DATA_post.dropna(how='all', axis=1, inplace=True)
len(ORG_DATA_post)

44744


42883

#### 2.D. Course List

In [10]:
# Course list
course_list['coursetitle'].value_counts()

# drop this as it is not important 
course_list = course_list[course_list['competencealias'] != 'LKT EL']

# Anywhere it says any variation of LKT - chnage it to LKT 
course_list['competencealias']= course_list['competencealias'].map(lambda x: "LKT" if "LKT" in x else x)

course_list = course_list[['typemeaning', 'competencealias', 'competence','coursetitle', 'proficiencylevel']]

#### 2.E. Recommendations

In [11]:
# Recommendations LOSC
losc_recom = losc_recom[['deliveryunit', 'workgroupset', 'competencealias',
       'parentcompetence', 'recommendednumberofpeopletotrain',
       'losccostbenefit(£)', 'competencedescription']]

# Recommendations FMS
fms_recom = fms_recom[['deliveryunit', 'workgroupset', 'competencealias','recommendednumberofpeopletotrain',
       'benefit(delayminutes)', 'competencedescription']]
# Recommendations Workbank
workbank_recom = workbank_recom[['deliveryunit', 'workgroupset', 'competencealias','recommendednumberofpeopletotrain',
       'workbankbenefit(backloghrs)', 'competencedescription']]

## 3. Extract Team Size from OrgPlus, Work group set info 

#### 3.A. WGS + Orgplus

In [12]:
def orgplus_wgs_merge(table_name1,table_name2):
    OrgPlus_WGS = pd.merge(table_name1,
                           table_name2[['routelookup','smengineerupn', 'workgroupset', 'masterdeliveryunit']],
                           left_on='parentupn', right_on='smengineerupn', how='left')

    # Drop all records with no parent UPN OrgPlus_WGS we can't link those to a Delivery Unit
    OrgPlus_WGS = OrgPlus_WGS.loc[~OrgPlus_WGS['smengineerupn'].isna()].reset_index(drop=True)

    OrgPlus_WGS = OrgPlus_WGS[['costcentre','routelookup',
        'employeenumber','parentupn', 'upn', 'smengineerupn', 'workgroupset',
        'masterdeliveryunit']]

    return OrgPlus_WGS


OrgPlus_WGS_pre = orgplus_wgs_merge(ORG_DATA_pre,workgroup_data_pre)
OrgPlus_WGS_post = orgplus_wgs_merge(ORG_DATA_post,workgroup_data_post)

#### 3.B. Team Size

In [32]:
def calculate_team_size(table_name):
    Team_size = table_name.groupby(['masterdeliveryunit', 'workgroupset'])['employeenumber'].nunique().reset_index()
    Team_size.rename(columns={'employeenumber':'Team Size'},inplace=True)
    return Team_size
team_size_pre = calculate_team_size(OrgPlus_WGS_pre)
team_size_post = calculate_team_size(OrgPlus_WGS_post)

# save output as csv and upload to azure blob storage containers
save_outputs_azure_blobs(team_size_pre,'team_size_PreInter.csv','teamSize',blob_service_client_instance_devp)
save_outputs_azure_blobs(team_size_post,'team_size_PostInter.csv','teamSize',blob_service_client_instance_devp)

save_outputs_azure_blobs(team_size_pre,'team_size_PreInter.csv','teamSize',blob_service_client_instance_test)
save_outputs_azure_blobs(team_size_post,'team_size_PostInter.csv','teamSize',blob_service_client_instance_test)

save_outputs_azure_blobs(team_size_pre,'team_size_PreInter.csv','teamSize',blob_service_client_instance_prod)
save_outputs_azure_blobs(team_size_post,'team_size_PostInter.csv','teamSize',blob_service_client_instance_prod)

save_outputs_azure_blobs(team_size_pre,'team_size_PreInter.csv','teamSize',blob_service_client_instance_staging)
save_outputs_azure_blobs(team_size_post,'team_size_PostInter.csv','teamSize',blob_service_client_instance_staging)

## 4. Merge trainning delivered with trainning recommendations

#### 4.A. Merge work group set info with trainning demand to assign employees to work group sets

In [20]:
# Merge work group set info with trainning demand to assign employees to work group sets
training_demand_orgplus_wgs = pd.merge(trainning_demand_filter,OrgPlus_WGS_post,how='left',
                                       left_on=['employeenumber'], 
                                       right_on=['employeenumber'])
training_demand_orgplus_wgs = training_demand_orgplus_wgs.loc[~training_demand_orgplus_wgs['workgroupset'].isna()].reset_index(drop=True)
training_demand_orgplus_wgs['workgroupset'].nunique()

2349

#### 4.B. Merge recommendations with courses and trainning delivered

In [21]:
# Merge losc recommendations with courses 
def trainning_delivered(table_name1):
    recom_course = pd.merge(table_name1,course_list,how='left',
                    left_on=['competencealias'],
                    right_on=['competencealias'])
    
    recomen_course_trainning = pd.merge(training_demand_orgplus_wgs,recom_course,how='left',
                                       left_on = ['coursetitle','workgroupset'],
                                       right_on=['coursetitle','workgroupset'])
    recomen_course_trainning = recomen_course_trainning.loc[~recomen_course_trainning['recommendednumberofpeopletotrain'].isna()].reset_index(drop=True)   
    print(len(recomen_course_trainning))
    return recomen_course_trainning

workbank_recomen_course_trainning = trainning_delivered(workbank_recom)
workbank_recomen_course_trainning = pd.DataFrame(workbank_recomen_course_trainning.groupby(['routelookup','workgroupset','masterdeliveryunit','coursetitle','competencealias','recommendednumberofpeopletotrain','workbankbenefit(backloghrs)'])['employeenumber'].count()).reset_index()

fms_recomen_course_trainning = trainning_delivered(fms_recom)
fms_recomen_course_trainning = pd.DataFrame(fms_recomen_course_trainning.groupby(['routelookup','workgroupset','masterdeliveryunit','coursetitle','competencealias','recommendednumberofpeopletotrain','benefit(delayminutes)'])['employeenumber'].count()).reset_index()

losc_recomen_course_trainning = trainning_delivered(losc_recom)
losc_recomen_course_trainning = pd.DataFrame(losc_recomen_course_trainning.groupby(['routelookup','workgroupset','masterdeliveryunit','coursetitle','competencealias','recommendednumberofpeopletotrain','losccostbenefit(£)'])['employeenumber'].count()).reset_index()


396
254
93


#### 4.C Save outputs to Azure blob storage

In [35]:

# Save outputs as csv and read to azure blob storage containers
save_outputs_azure_blobs(workbank_recomen_course_trainning,'workbank_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_devp)
save_outputs_azure_blobs(fms_recomen_course_trainning,'fms_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_devp)
save_outputs_azure_blobs(losc_recomen_course_trainning,'losc_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_devp)

# Save outputs as csv and read to azure blob storage containers
save_outputs_azure_blobs(workbank_recomen_course_trainning,'workbank_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_test)
save_outputs_azure_blobs(fms_recomen_course_trainning,'fms_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_test)
save_outputs_azure_blobs(losc_recomen_course_trainning,'losc_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_test)

# Save outputs as csv and read to azure blob storage containers
save_outputs_azure_blobs(workbank_recomen_course_trainning,'workbank_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_prod)
save_outputs_azure_blobs(fms_recomen_course_trainning,'fms_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_prod)
save_outputs_azure_blobs(losc_recomen_course_trainning,'losc_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_prod)

# Save outputs as csv and read to azure blob storage containers
save_outputs_azure_blobs(workbank_recomen_course_trainning,'workbank_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_staging)
save_outputs_azure_blobs(fms_recomen_course_trainning,'fms_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_staging)
save_outputs_azure_blobs(losc_recomen_course_trainning,'losc_recomen_course_trainning.csv','trainning_delivered_recommendations',blob_service_client_instance_staging)